# Getting all available Swiss petrol stations using the overpass Web API

## Libraries and settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/Users/sivanujanselvarajah/Documents/zhaw/4.Semester/Scientific programming/scientific_programming/Week_04/challenge


## Overpass turbo query to get available petrol stations

In [2]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["amenity"="fuel"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to .json file
with open('petrol_stations.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = pd.json_normalize(data)
df.head()

,type,id,lat,lon,tags.amenity,tags.fixme,tags.name,tags.operator,tags.brand,tags.fuel:diesel,...,tags.payment:post_finance,tags.covered,tags.payment:girocard,tags.payment:vpay,tags.fuel:diesel_plus,tags.construction,tags.dog,tags.payment:hubrol_card,tags.fuel:lng,tags.fuel:petrol
0,node,734402,46.690599,7.668189,fuel,Please check name and brand. Agip becomes Eni....,Agip,Agip,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,1228781,46.878598,7.542326,fuel,NaN,NaN,BP,BP,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,5702030,46.680546,7.827412,fuel,NaN,Migrol,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,5702071,46.683415,7.844559,fuel,NaN,Avia,Avia,Avia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,8082505,46.683817,7.664427,fuel,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save data frame to a .csv file

In [7]:
# Write data frame to .csv - file
df.to_csv('petrol-stations.csv', index=False, sep=";", encoding='utf-8')

## Use a pivot table to count the number of petrol stations per operator

In [4]:
# Create pivot table
tab = pd.pivot_table(df[['id', 'tags.name']], 
                       values=['id'], 
                       index=['tags.name'],
                       aggfunc='count')

# Sort pivot table
tab_sorted = tab.sort_values(by=['id'], ascending=False)
tab_sorted

,id
tags.name,
Avia,162
Agrola,148
BP,110
Tamoil,97
Shell,77
...,...
Garage du Clos,1
Garage du Gros-de-Vaud,1
Garage du Nord,1


## Plot petrol stations on map

In [8]:
# Subset of petrol stations by brand
locations = df[["lat", 
                "lon", 
                "tags.brand", 
                "tags.shop"]].loc[df["tags.brand"].isin(['Migrol', 
                                                         'Agip', 
                                                         'Shell'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                   popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

          lat       lon tags.brand tags.shop
8   47.686119  8.621418      Shell       NaN
19  47.495696  8.712981      Shell       NaN
23  47.381593  8.509545      Shell       NaN
28  47.381110  8.492148     Migrol       NaN
32  47.337432  8.530242      Shell       NaN


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [10]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.3.0
Datetime: 2024-03-23 11:21:25
Python Version: 3.10.13
-----------------------------------
